##1 | Importing Libraries

In [ ]:
# Importing Libraries

from warnings import filterwarnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import string
import nltk
from nltk.tokenize import word_tokenize
from PIL import Image
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate
from sklearn.preprocessing import LabelEncoder
from textblob import Word, TextBlob
from wordcloud import WordCloud


##2 | Data Processing

In [ ]:
#mount google drive

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Importing and Reading Metadata JSON File
metadata_json_file = '/content/drive/MyDrive/Amazon Metadata/meta_Video_Games.json'
df1 = pd.read_json(metadata_json_file,convert_dates=True,lines=True,orient='columns')
df1.head(10)

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"[Video Games, PC, Games]",,[],,Reversi Sensory Challenger,[],,Fidelity Electronics,[],"[>#2,623,937 in Toys &amp; Games (See Top 100 ...",[],Toys &amp; Games,,,,0042000742,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
1,"[Video Games, Xbox 360, Games, </span></span><...",,[Brand new sealed!],,Medal of Honor: Warfighter - Includes Battlefi...,[B00PADROYW],,by\n \n EA Games,[],"[>#67,231 in Video Games (See Top 100 in Video...","[B0050SY5BM, B072NQJCW5, B000TI836G, B002SRSQ7...",Video Games,,,"\n\t\t\t\t\t\t\t\t\t\t\t\t<span class=""vertica...",0078764343,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
2,"[Video Games, Retro Gaming & Microconsoles, Su...",,[],,street fighter 2 II turbo super nintendo snes ...,[],,Nintendo,[],"[>#134,433 in Video Games (See Top 100 in Vide...",[],Video Games,,,$0.72,0276425316,[],[],NaN
3,"[Video Games, Xbox 360, Accessories, Controlle...",,[MAS's Pro Xbox 360 Stick (Perfect 360 Stick) ...,,Xbox 360 MAS STICK,[],,by\n \n MAS SYSTEMS,[Original PCB used from Xbox 360 Control Pad (...,"[>#105,263 in Video Games (See Top 100 in Vide...",[],Video Games,,,,0324411812,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
4,"[Video Games, PC, Games, </span></span></span>...",,"[Phonics Alive! 3, The Speller teaches student...",,Phonics Alive! 3: The Speller,[],,by\n \n Advanced Software Pty. Ltd.,"[Grades 2-12, Spelling Program, Teaches Spelli...","[>#92,397 in Video Games (See Top 100 in Video...",[B000BCZ7U0],Video Games,,,,0439335310,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84814,"[Video Games, PlayStation Vita, Digital Games ...",,[<div>The Force is strong with this one The No...,,Lego Star Wars: The Force Awakens - PS Vita [D...,[],,by\n \n Warner Bros.,[],"[>#74,224 in Video Games (See Top 100 in Video...",[],Video Games,,,,B01HJ1521Y,[],[],{}
84815,"[Video Games, PlayStation 4, Digital Games & D...",,[<div>The Season Pass includes three Level Pac...,,Lego Star Wars: The Force Awakens Season Pass...,[],,by\n \n Warner Bros.,[DLC Requires base game],"[>#62,150 in Video Games (See Top 100 in Video...",[],Video Games,,,\n\t\t ...,B01HJ14TTA,[],[],{}
84816,"[Video Games, PlayStation 4, Digital Games & D...",,"[<div>The Technomancer takes you to Mars, wher...",,The Technomancer - PS4 [Digital Code],[],,by\n \n Focus Home Interactive,[],"[>#94,234 in Video Games (See Top 100 in Video...",[],Video Games,,,,B01HJ14OT0,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}
84817,"[Video Games, Xbox 360, Accessories, </span></...",,[<b>FUNCTIONS:</b><br> 1.Take apart your Xbox ...,,"Repair T8 T6 Tools for XBOX One Xbox 360, YTTL...","[B01KBNB7K2, B06X6JSYPC, B01N6Y0Z7W, B06VXD2W5...",,by\n \n YTTL,[If you want to Replacement you Xbox one /360 ...,"[>#16,087 in Video Games (See Top 100 in Video...","[B01KH25ZY6, B00PG8SU26, B07G122BVS, B016XLTQP...",Video Games,,,"\n\t\t\t\t\t\t\t\t\t\t\t\t<span class=""vertica...",B01HJC33WS,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}


In [ ]:
# Read Reviews JSON file line by line and store as a list of dictionaries
json_data = []
with open("/content/drive/MyDrive/Amazon Metadata/Video_Games.json") as file:
    for line in file:
        json_data.append(json.loads(line))

# Create DataFrame
df2 = pd.DataFrame(json_data)
df2.head(10)

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,1.0,True,"06 9, 2014",A21ROB4YDOZA5P,0439381673,Mary M. Clark,I used to play this game years ago and loved i...,Did not like this,1402272000,NaN,NaN,NaN
1,3.0,True,"05 10, 2014",A3TNZ2Q5E7HTHD,0439381673,Sarabatya,The game itself worked great but the story lin...,Almost Perfect,1399680000,NaN,NaN,NaN
2,4.0,True,"02 7, 2014",A1OKRM3QFEATQO,0439381673,Amazon Customer,I had to learn the hard way after ordering thi...,DOES NOT WORK WITH MAC OS unless it is 10.3 or...,1391731200,15,NaN,NaN
3,1.0,True,"02 7, 2014",A2XO1JFCNEYV3T,0439381673,ColoradoPartyof5,The product description should state this clea...,does not work on Mac OSX,1391731200,11,NaN,NaN
4,4.0,True,"01 16, 2014",A19WLPIRHD15TH,0439381673,Karen Robinson,I would recommend this learning game for anyon...,Roughing it,1389830400,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2565344,5.0,True,"08 1, 2018",ANGB54K3888S4,B01HJEBIAA,josh,"Love it, work good",Works good,1533081600,NaN,NaN,NaN
2565345,5.0,True,"07 17, 2018",A3TEVKR0ZVQB2T,B01HJEBIAA,Prime Member,I do a lot of copy/paste and other keyboard sh...,Great mouse for work and gaming,1531785600,NaN,NaN,[https://images-na.ssl-images-amazon.com/image...
2565346,5.0,True,"07 6, 2018",ABE7YPWEHNVJZ,B01HJEBIAA,Amazon Customer,One year in and it's still working great! Hig...,Five Stars,1530835200,NaN,NaN,NaN
2565347,5.0,True,"06 12, 2018",A3ES9QBK3G192O,B01HJEBIAA,Lina Marmolejos,EXCELENTE,Five Stars,1528761600,NaN,NaN,NaN


In [ ]:
# Merge DataFrames based on asin
merged_df = pd.merge(df1, df2, on='asin', how='outer')
merged_df.head(10)

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,...,verified,reviewTime,reviewerID,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,"[Video Games, PC, Games]",,[],,Reversi Sensory Challenger,[],,Fidelity Electronics,[],"[>#2,623,937 in Toys &amp; Games (See Top 100 ...",...,True,"03 11, 2015",A285UYK35TF093,Stefano,Simply extraordinary! A jump into my past and ...,Top of the top Reversi Machine,1.426032e+09,NaN,NaN,NaN
1,"[Video Games, Xbox 360, Games, </span></span><...",,[Brand new sealed!],,Medal of Honor: Warfighter - Includes Battlefi...,[B00PADROYW],,by\n \n EA Games,[],"[>#67,231 in Video Games (See Top 100 in Video...",...,False,"09 4, 2014",A38A8ACEHLMQ9X,juan arango,Good product.\nZero Problem.,Five Stars,1.409789e+09,NaN,NaN,NaN
2,"[Video Games, Xbox 360, Games, </span></span><...",,[Brand new sealed!],,Medal of Honor: Warfighter - Includes Battlefi...,[B00PADROYW],,by\n \n EA Games,[],"[>#67,231 in Video Games (See Top 100 in Video...",...,True,"08 24, 2013",A24SSUT5CSW8BH,Kindle Customer,I want to start off by saying I have never pla...,Love the game,1.377302e+09,NaN,NaN,NaN
3,"[Video Games, Xbox 360, Games, </span></span><...",,[Brand new sealed!],,Medal of Honor: Warfighter - Includes Battlefi...,[B00PADROYW],,by\n \n EA Games,[],"[>#67,231 in Video Games (See Top 100 in Video...",...,True,"07 7, 2013",AB9S9279OZ3QO,Alan,I haven't gotten around to playing the campaig...,Good game and Beta access!!,1.373155e+09,2,NaN,NaN
4,"[Video Games, Xbox 360, Games, </span></span><...",,[Brand new sealed!],,Medal of Honor: Warfighter - Includes Battlefi...,[B00PADROYW],,by\n \n EA Games,[],"[>#67,231 in Video Games (See Top 100 in Video...",...,False,"07 4, 2013",AK3V0HEBJMQ7J,Royalty,this will be my second medal of honor I love h...,MOH nice,1.372896e+09,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2848243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,"09 21, 2015",A3FLXBA8OTBECJ,Skinny J,"I Love this game, it was my childhood game whe...",I Love this game,1.442794e+09,NaN,NaN,NaN
2848244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,"01 5, 2012",A2OC3494D0S0EQ,Kindle Customer,I have windows 7 and despite my best efforts a...,Don't recommend,1.325722e+09,NaN,NaN,NaN
2848245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,"01 1, 2012",AND4U918A3WXX,elviscat,game is too old. I was going to have to jump t...,wouldn't download on my newer computer,1.325376e+09,NaN,NaN,NaN
2848246,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,"04 15, 2009",A34NS5U78IRBQD,I. Lim,"On starting the game, a tutorial walks you thr...",Cooking game with a new twist,1.239754e+09,4,NaN,NaN


In [ ]:
df3 = merged_df[['category', 'asin', 'title', 'reviewText']]

In [ ]:
df4 = df3.dropna()
df4.head(10)

,category,asin,title,reviewText
0,"[Video Games, PC, Games]",0042000742,Reversi Sensory Challenger,Simply extraordinary! A jump into my past and ...
1,"[Video Games, Xbox 360, Games, </span></span><...",0078764343,Medal of Honor: Warfighter - Includes Battlefi...,Good product.\nZero Problem.
2,"[Video Games, Xbox 360, Games, </span></span><...",0078764343,Medal of Honor: Warfighter - Includes Battlefi...,I want to start off by saying I have never pla...
3,"[Video Games, Xbox 360, Games, </span></span><...",0078764343,Medal of Honor: Warfighter - Includes Battlefi...,I haven't gotten around to playing the campaig...
4,"[Video Games, Xbox 360, Games, </span></span><...",0078764343,Medal of Honor: Warfighter - Includes Battlefi...,this will be my second medal of honor I love h...
...,...,...,...,...
2844567,"[Video Games, PC, Accessories, Gaming Mice]",B01HJEBIAA,Zelotes 4000 DPI Programmable Gaming Mouse for...,"Love it, work good"
2844568,"[Video Games, PC, Accessories, Gaming Mice]",B01HJEBIAA,Zelotes 4000 DPI Programmable Gaming Mouse for...,I do a lot of copy/paste and other keyboard sh...
2844569,"[Video Games, PC, Accessories, Gaming Mice]",B01HJEBIAA,Zelotes 4000 DPI Programmable Gaming Mouse for...,One year in and it's still working great! Hig...
2844570,"[Video Games, PC, Accessories, Gaming Mice]",B01HJEBIAA,Zelotes 4000 DPI Programmable Gaming Mouse for...,EXCELENTE


In [ ]:
#create a summary table for count of review on the basis of Asin id
asin_review_count = df3.groupby(['asin','title'])['reviewText'].count().reset_index(name='review_count')

In [ ]:
asin_review_count

,asin,title,review_count
0,0042000742,Reversi Sensory Challenger,1
1,0078764343,Medal of Honor: Warfighter - Includes Battlefi...,4
2,0276425316,street fighter 2 II turbo super nintendo snes ...,1
3,0324411812,Xbox 360 MAS STICK,1
4,0439335299,Phonics Alive! 2 The Sound Blender,1
...,...,...,...
71906,B01HJ14OT0,The Technomancer - PS4 [Digital Code],1
71907,B01HJ14TTA,Lego Star Wars: The Force Awakens Season Pass...,4
71908,B01HJ1521Y,Lego Star Wars: The Force Awakens - PS Vita [D...,1
71909,B01HJC33WS,"Repair T8 T6 Tools for XBOX One Xbox 360, YTTL...",22


##3 | Text Preprocessing

####3.1. | Normalizing Case Folding
In the first step of our project, we converted the comments in the 'reviewText' column to lowercase. This standardizes the data into a uniform format, eliminating inconsistencies that might arise from differences in case sensitivity during the text processing phase.

####3.2 | Punctuations
In this step, we removed punctuation marks from the comments in the 'reviewText' column. This process aims to enhance the efficiency of our language processing tasks by making our text data cleaner and more suitable for analysis.

####3.3 | Stopwords
In this section, by removing frequently repeated and often meaningless words (such as 'the', 'is', 'in') from our texts, we can focus our analysis on more meaningful words. This allows us to better understand the essence of the comments and make our processes more efficient.

####3.4 | Lemmatization
In this step, we lemmatized each word in the comments of the 'reviewText' column to obtain their root forms (basic forms). Lemmatization reduces different inflected forms of words (like 'running' to 'run') to their base form. This provides a more consistent and meaningful data processing opportunity in our analyses.

####3.5 | Tokenization
In this part, we transformed each comment in the 'reviewText' column into a TextBlob object and then tokenized them into words. TextBlob allows us to easily split comments into processable tokens (words). This process enables us to examine the words in each comment individually and conduct more detailed analyses based on these words.

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

def tokenize_text(text):
    # Check if input text is not a string, convert it to string
    if not isinstance(text, str):
        text = str(text)

    # Tokenize the text into lower words
    tokens = word_tokenize(text.lower())

    # Remove punctuation tokens
    tokens = [token for token in tokens if token not in string.punctuation]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Join tokens into a single string
    return ' '.join(tokens)

In [ ]:
# Remove tokens from 'reviewText' column
df4['reviewText'] = df4['reviewText'].apply(tokenize_text)

# Now, 'reviewText' column contains cleaned text without tokens
# You can proceed with sentiment analysis using this cleaned text data
df4

<ipython-input-12-abcbbb3376ba>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['reviewText'] = df4['reviewText'].apply(tokenize_text)


,category,asin,title,reviewText
0,"[Video Games, PC, Games]",0042000742,Reversi Sensory Challenger,simply extraordinary jump past sound late seve...
1,"[Video Games, Xbox 360, Games, </span></span><...",0078764343,Medal of Honor: Warfighter - Includes Battlefi...,good product zero problem
2,"[Video Games, Xbox 360, Games, </span></span><...",0078764343,Medal of Honor: Warfighter - Includes Battlefi...,want start saying never played call duty game ...
3,"[Video Games, Xbox 360, Games, </span></span><...",0078764343,Medal of Honor: Warfighter - Includes Battlefi...,n't gotten around playing campaign multiplayer...
4,"[Video Games, Xbox 360, Games, </span></span><...",0078764343,Medal of Honor: Warfighter - Includes Battlefi...,second medal honor love incorporate real life ...
...,...,...,...,...
2844567,"[Video Games, PC, Accessories, Gaming Mice]",B01HJEBIAA,Zelotes 4000 DPI Programmable Gaming Mouse for...,love work good
2844568,"[Video Games, PC, Accessories, Gaming Mice]",B01HJEBIAA,Zelotes 4000 DPI Programmable Gaming Mouse for...,lot copy/paste keyboard shortcut job wanted so...
2844569,"[Video Games, PC, Accessories, Gaming Mice]",B01HJEBIAA,Zelotes 4000 DPI Programmable Gaming Mouse for...,one year 's still working great highly recomme...
2844570,"[Video Games, PC, Accessories, Gaming Mice]",B01HJEBIAA,Zelotes 4000 DPI Programmable Gaming Mouse for...,excelente


In [ ]:
df4.to_csv('Video_Games.csv', index=False)

In [ ]:
df4 = pd.read_csv('Video_Games.csv')

In [ ]:
df4.head(10)

,category,asin,title,reviewText
0,"['Video Games', 'PC', 'Games']",0042000742,Reversi Sensory Challenger,simply extraordinary jump past sound late seve...
1,"['Video Games', 'Xbox 360', 'Games', '</span><...",0078764343,Medal of Honor: Warfighter - Includes Battlefi...,good product zero problem
2,"['Video Games', 'Xbox 360', 'Games', '</span><...",0078764343,Medal of Honor: Warfighter - Includes Battlefi...,want start saying never played call duty game ...
3,"['Video Games', 'Xbox 360', 'Games', '</span><...",0078764343,Medal of Honor: Warfighter - Includes Battlefi...,n't gotten around playing campaign multiplayer...
4,"['Video Games', 'Xbox 360', 'Games', '</span><...",0078764343,Medal of Honor: Warfighter - Includes Battlefi...,second medal honor love incorporate real life ...
5,"['Video Games', 'Retro Gaming & Microconsoles'...",0276425316,street fighter 2 II turbo super nintendo snes ...,good product good condition thanks
6,"['Video Games', 'Xbox 360', 'Accessories', 'Co...",0324411812,Xbox 360 MAS STICK,ordered arcade stick received business law tex...
7,"['Video Games', 'PC', 'Games', '</span></span>...",0439335310,Phonics Alive! 3: The Speller,great information old use newer window system
8,"['Video Games', 'PC', 'Games', '</span></span>...",0439339006,Sim City 3000,didnt really think would issue turn game wo n'...
9,"['Video Games', 'PC', 'Games', '</span></span>...",0439339006,Sim City 3000,n't work window 10. make clear description


In [ ]:
# Import the 're' module for regular expressions if not already imported
import re

# Ensure that the 'category' column is of string type
df4['category'] = df4['category'].astype(str)

# Split the 'category' column using a comma and extract the first part into 'category1'
df4["category1"] = df4['category'].str.split(',',1).str[0]

# Split the 'category' column using a comma and extract the second part into 'category2'
df4["category2"] = df4['category'].str.split(',',2).str[1]

# Split the 'category' column using a comma and extract the third part into 'category3'
df4["category3"] = df4['category'].str.split(',',3).str[2]

# Remove any square brackets and single quotes from 'category1'
df4['category1'] = df4['category1'].astype(str)
df4['category1'] = df4['category1'].apply(lambda x: re.sub(r"[\[\]']", "", x))

# Remove any single quotes from 'category2'
df4['category2'] = df4['category2'].astype(str)
df4['category2'] = df4['category2'].apply(lambda x: re.sub(r"'", "", x))

# Remove any square brackets and single quotes from 'category3'
df4['category3'] = df4['category3'].astype(str)
df4['category3'] = df4['category3'].apply(lambda x: re.sub(r"[\[\]']", "", x))

# Drop the original 'rank' column
df4 = df4.drop(columns='category')

# Display the modified DataFrame
df4

<ipython-input-18-612d817b8c9e>:8: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df4["category1"] = df4['category'].str.split(',',1).str[0]
<ipython-input-18-612d817b8c9e>:11: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df4["category2"] = df4['category'].str.split(',',2).str[1]
<ipython-input-18-612d817b8c9e>:14: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df4["category3"] = df4['category'].str.split(',',3).str[2]


,asin,title,reviewText,category1,category2,category3
0,0042000742,Reversi Sensory Challenger,simply extraordinary jump past sound late seve...,Video Games,PC,Games
1,0078764343,Medal of Honor: Warfighter - Includes Battlefi...,good product zero problem,Video Games,Xbox 360,Games
2,0078764343,Medal of Honor: Warfighter - Includes Battlefi...,want start saying never played call duty game ...,Video Games,Xbox 360,Games
3,0078764343,Medal of Honor: Warfighter - Includes Battlefi...,n't gotten around playing campaign multiplayer...,Video Games,Xbox 360,Games
4,0078764343,Medal of Honor: Warfighter - Includes Battlefi...,second medal honor love incorporate real life ...,Video Games,Xbox 360,Games
...,...,...,...,...,...,...
2842753,B01HJEBIAA,Zelotes 4000 DPI Programmable Gaming Mouse for...,love work good,Video Games,PC,Accessories
2842754,B01HJEBIAA,Zelotes 4000 DPI Programmable Gaming Mouse for...,lot copy/paste keyboard shortcut job wanted so...,Video Games,PC,Accessories
2842755,B01HJEBIAA,Zelotes 4000 DPI Programmable Gaming Mouse for...,one year 's still working great highly recomme...,Video Games,PC,Accessories
2842756,B01HJEBIAA,Zelotes 4000 DPI Programmable Gaming Mouse for...,excelente,Video Games,PC,Accessories


##4 | Sentiment Analysis

In [ ]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
df4["reviewText"].head()

0    simply extraordinary jump past sound late seve...
1                            good product zero problem
2    want start saying never played call duty game ...
3    n't gotten around playing campaign multiplayer...
4    second medal honor love incorporate real life ...
Name: reviewText, dtype: object

In [ ]:
sia = SentimentIntensityAnalyzer()

In [ ]:
df4["reviewText"][0:10].apply(lambda x: sia.polarity_scores(x))

0    {'neg': 0.0, 'neu': 0.419, 'pos': 0.581, 'comp...
1    {'neg': 0.355, 'neu': 0.263, 'pos': 0.382, 'co...
2    {'neg': 0.035, 'neu': 0.646, 'pos': 0.32, 'com...
3    {'neg': 0.144, 'neu': 0.481, 'pos': 0.375, 'co...
4    {'neg': 0.0, 'neu': 0.324, 'pos': 0.676, 'comp...
5    {'neg': 0.0, 'neu': 0.187, 'pos': 0.813, 'comp...
6    {'neg': 0.0, 'neu': 0.796, 'pos': 0.204, 'comp...
7    {'neg': 0.0, 'neu': 0.594, 'pos': 0.406, 'comp...
8    {'neg': 0.177, 'neu': 0.697, 'pos': 0.126, 'co...
9    {'neg': 0.0, 'neu': 0.698, 'pos': 0.302, 'comp...
Name: reviewText, dtype: object

In [ ]:
df4["reviewText"][0:10].apply(lambda x: sia.polarity_scores(x)["compound"])

0    0.9100
1    0.0516
2    0.9409
3    0.7723
4    0.9393
5    0.8271
6    0.3182
7    0.6249
8   -0.0444
9    0.3818
Name: reviewText, dtype: float64

In [ ]:
df4["reviewText"] = df4["reviewText"].astype(str)

In [ ]:
df4["polarity_score"] = df4["reviewText"].apply(lambda x: sia.polarity_scores(x)["compound"])

In [ ]:
df4

,asin,title,reviewText,category1,category2,category3,polarity_score
0,0042000742,Reversi Sensory Challenger,simply extraordinary jump past sound late seve...,Video Games,PC,Games,0.9100
1,0078764343,Medal of Honor: Warfighter - Includes Battlefi...,good product zero problem,Video Games,Xbox 360,Games,0.0516
2,0078764343,Medal of Honor: Warfighter - Includes Battlefi...,want start saying never played call duty game ...,Video Games,Xbox 360,Games,0.9409
3,0078764343,Medal of Honor: Warfighter - Includes Battlefi...,n't gotten around playing campaign multiplayer...,Video Games,Xbox 360,Games,0.7723
4,0078764343,Medal of Honor: Warfighter - Includes Battlefi...,second medal honor love incorporate real life ...,Video Games,Xbox 360,Games,0.9393
...,...,...,...,...,...,...,...
2842753,B01HJEBIAA,Zelotes 4000 DPI Programmable Gaming Mouse for...,love work good,Video Games,PC,Accessories,0.7964
2842754,B01HJEBIAA,Zelotes 4000 DPI Programmable Gaming Mouse for...,lot copy/paste keyboard shortcut job wanted so...,Video Games,PC,Accessories,0.8555
2842755,B01HJEBIAA,Zelotes 4000 DPI Programmable Gaming Mouse for...,one year 's still working great highly recomme...,Video Games,PC,Accessories,0.7346
2842756,B01HJEBIAA,Zelotes 4000 DPI Programmable Gaming Mouse for...,excelente,Video Games,PC,Accessories,0.0000


In [ ]:
#export df4 into json
#df4.to_json(r'/content/drive/MyDrive/Amazon Metadata/Video_Games_with_Score.json', orient='columns')

In [ ]:
df5 = df4[['asin', 'title', 'category1', 'category2', 'category3', 'polarity_score']]

In [ ]:
df5

,asin,title,category1,category2,category3,polarity_score
0,0042000742,Reversi Sensory Challenger,Video Games,PC,Games,0.9100
1,0078764343,Medal of Honor: Warfighter - Includes Battlefi...,Video Games,Xbox 360,Games,0.0516
2,0078764343,Medal of Honor: Warfighter - Includes Battlefi...,Video Games,Xbox 360,Games,0.9409
3,0078764343,Medal of Honor: Warfighter - Includes Battlefi...,Video Games,Xbox 360,Games,0.7723
4,0078764343,Medal of Honor: Warfighter - Includes Battlefi...,Video Games,Xbox 360,Games,0.9393
...,...,...,...,...,...,...
2842753,B01HJEBIAA,Zelotes 4000 DPI Programmable Gaming Mouse for...,Video Games,PC,Accessories,0.7964
2842754,B01HJEBIAA,Zelotes 4000 DPI Programmable Gaming Mouse for...,Video Games,PC,Accessories,0.8555
2842755,B01HJEBIAA,Zelotes 4000 DPI Programmable Gaming Mouse for...,Video Games,PC,Accessories,0.7346
2842756,B01HJEBIAA,Zelotes 4000 DPI Programmable Gaming Mouse for...,Video Games,PC,Accessories,0.0000


In [ ]:
# prompt: join df5 with df1['overall']

df6 = df5.join(df2['overall'], how='left')


In [ ]:
df6

,asin,title,category1,category2,category3,polarity_score,overall
0,0042000742,Reversi Sensory Challenger,Video Games,PC,Games,0.9100,1.0
1,0078764343,Medal of Honor: Warfighter - Includes Battlefi...,Video Games,Xbox 360,Games,0.0516,3.0
2,0078764343,Medal of Honor: Warfighter - Includes Battlefi...,Video Games,Xbox 360,Games,0.9409,4.0
3,0078764343,Medal of Honor: Warfighter - Includes Battlefi...,Video Games,Xbox 360,Games,0.7723,1.0
4,0078764343,Medal of Honor: Warfighter - Includes Battlefi...,Video Games,Xbox 360,Games,0.9393,4.0
...,...,...,...,...,...,...,...
2842753,B01HJEBIAA,Zelotes 4000 DPI Programmable Gaming Mouse for...,Video Games,PC,Accessories,0.7964,NaN
2842754,B01HJEBIAA,Zelotes 4000 DPI Programmable Gaming Mouse for...,Video Games,PC,Accessories,0.8555,NaN
2842755,B01HJEBIAA,Zelotes 4000 DPI Programmable Gaming Mouse for...,Video Games,PC,Accessories,0.7346,NaN
2842756,B01HJEBIAA,Zelotes 4000 DPI Programmable Gaming Mouse for...,Video Games,PC,Accessories,0.0000,NaN


In [ ]:
org_data = df6.groupby(['asin', 'title', 'category1', 'category2', 'category3'])['overall'].mean()

In [ ]:
org_data

asin        title                                                                                                                                                                       category1    category2                      category3           
0042000742  Reversi Sensory Challenger                                                                                                                                                  Video Games   PC                             Games                  1.0
0078764343  Medal of Honor: Warfighter - Includes Battlefield 4 Beta - Limited Edition                                                                                                  Video Games   Xbox 360                       Games                  3.0
0276425316  street fighter 2 II turbo super nintendo snes super nes video game                                                                                                          Video Games   Retro Gaming & Microconsoles   Super Nint

In [ ]:
org_data.to_csv('org_data.csv')

In [ ]:
# Assuming df5 is your DataFrame
# Group by 'asin' and calculate the mean polarity score
average_polarity_by_asin = df5.groupby(['asin', 'title', 'category1', 'category2', 'category3'])['polarity_score'].mean()

In [ ]:
average_polarity_by_asin

asin        title                                                                                                                                                                       category1    category2                      category3           
0042000742  Reversi Sensory Challenger                                                                                                                                                  Video Games   PC                             Games                  0.910000
0078764343  Medal of Honor: Warfighter - Includes Battlefield 4 Beta - Limited Edition                                                                                                  Video Games   Xbox 360                       Games                  0.676025
0276425316  street fighter 2 II turbo super nintendo snes super nes video game                                                                                                          Video Games   Retro Gaming & Microconsoles   

In [ ]:
average_polarity_by_asin.to_csv('processed_data.csv')